In [27]:
# import torch
# import torchvision
import torchvision.transforms as transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [12]:
#load data
#this might take a while as it will download the dataset from internet
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
data_train = torchvision.datasets.MNIST('./', download=True, train=True, transform = transform)
data_test = torchvision.datasets.MNIST('./', download=True, train=False, transform = transform)

In [13]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(data_train, batch_size=batch_size)
test_dataloader = DataLoader(data_test, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [14]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [33]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303919  [    0/60000]
loss: 2.293607  [ 6400/60000]
loss: 2.271937  [12800/60000]
loss: 2.250540  [19200/60000]
loss: 2.227844  [25600/60000]
loss: 2.206530  [32000/60000]
loss: 2.189018  [38400/60000]
loss: 2.192962  [44800/60000]
loss: 2.175526  [51200/60000]
loss: 2.127569  [57600/60000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 0.032973 

Epoch 2
-------------------------------
loss: 2.080045  [    0/60000]
loss: 2.035159  [ 6400/60000]
loss: 2.082289  [12800/60000]
loss: 1.993146  [19200/60000]
loss: 1.972727  [25600/60000]
loss: 1.953683  [32000/60000]
loss: 1.922680  [38400/60000]
loss: 1.940952  [44800/60000]
loss: 1.919456  [51200/60000]
loss: 1.840536  [57600/60000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.027840 

Epoch 3
-------------------------------
loss: 1.725239  [    0/60000]
loss: 1.646641  [ 6400/60000]
loss: 1.794473  [12800/60000]
loss: 1.618450  [19200/60000]
loss: 1.597751  [25600/60000]
loss: 1.613489  [32000/600

In [41]:
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
]

model.eval()
for t in range(len(data_test)):
    x, y = data_test[t][0], data_test[t][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "7", Actual: "7"
Predicted: "6", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "6", Actual: "6"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "7", Actual: "9"
Predicted: "6", Actual: "6"
Predicted: "4", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "6", Actual: "4"
Predicted: "7", Actual: "7"
Predicted: "5", Actu

Predicted: "7", Actual: "7"
Predicted: "1", Actual: "1"
Predicted: "1", Actual: "1"
Predicted: "1", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "5", Actual: "5"
Predicted: "3", Actual: "3"
Predicted: "3", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "8"
Predicted: "6", Actual: "6"
Predicted: "3", Actual: "5"
Predicted: "4", Actual: "6"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "5", Actual: "8"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "5"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "3"
Predicted: "1", Actual: "1"
Predicted: "5", Actual: "5"
Predicted: "0", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "1", Actual: "1"
Predicted: "1", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "6", Actual: "6"
Predicted: "7", Actual: "2"
Predicted: "1", Actu

Predicted: "1", Actual: "1"
Predicted: "1", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "3", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "7"
Predicted: "7", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "5", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "5", Actual: "8"
Predicted: "6", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "4", Actual: "9"
Predicted: "3", Actual: "5"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "6", Actual: "6"
Predicted: "9", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "1", Actual: "7"
Predicted: "1", Actual: "8"
Predicted: "3", Actual: "3"
Predicted: "3", Actu

Predicted: "5", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "2"
Predicted: "5", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "8"
Predicted: "6", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "2"
Predicted: "3", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "3", Actual: "3"
Predicted: "9", Actual: "8"
Predicted: "3", Actual: "3"
Predicted: "5", Actual: "5"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "3"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "8"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "2"
Predicted: "9", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "6", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "9", Actu

Predicted: "6", Actual: "6"
Predicted: "6", Actual: "6"
Predicted: "5", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "5"
Predicted: "3", Actual: "8"
Predicted: "3", Actual: "2"
Predicted: "9", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "6", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "5", Actual: "0"
Predicted: "5", Actual: "5"
Predicted: "5", Actual: "5"
Predicted: "6", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "2"
Predicted: "6", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "6", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "4", Actu

...

Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "6"
Predicted: "6", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "3"
Predicted: "3", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "1", Actual: "1"
Predicted: "6", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "7", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "6", Actual: "8"
Predicted: "9", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "9", Actual: "8"
Predicted: "6", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "4", Actual: "5"
Predicted: "5", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "6", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "5", Actu

Predicted: "9", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "6", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "3", Actual: "3"
Predicted: "5", Actual: "5"
Predicted: "3", Actual: "3"
Predicted: "1", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "0"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "6", Actual: "5"
Predicted: "3", Actual: "3"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "8"
Predicted: "3", Actual: "3"
Predicted: "0", Actual: "0"
Predicted: "6", Actual: "6"
Predicted: "6", Actual: "2"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "8"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "3"
Predicted: "3", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "4", Actual: "4"
Predicted: "6", Actual: "2"
Predicted: "0", Actu

Predicted: "7", Actual: "7"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "3", Actual: "2"
Predicted: "6", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "6", Actual: "6"
Predicted: "3", Actual: "8"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "0"
Predicted: "6", Actual: "6"
Predicted: "1", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "5"
Predicted: "3", Actual: "8"
Predicted: "4", Actual: "6"
Predicted: "4", Actual: "2"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "3", Actual: "8"
Predicted: "5", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "7"
Predicted: "5", Actu

Predicted: "1", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "5", Actual: "6"
Predicted: "1", Actual: "1"
Predicted: "1", Actual: "2"
Predicted: "9", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "1", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "5", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "8"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "1", Actual: "8"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "2"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "6", Actual: "5"
Predicted: "6", Actual: "6"
Predicted: "7", Actu

Predicted: "5", Actual: "5"
Predicted: "6", Actual: "6"
